# Adaptive Cards Toolkit: 05e - Specialized Content Templates

This notebook demonstrates how to create specialized content templates like documentation pages and news digests for Adaptive Cards. It continues from where 05d_toolkit_profiles.ipynb left off. Please run 01_toolkit_common.ipynb first.

## Setup

First, we need to import the common utilities defined in the toolkit_common notebook as well as all necessary dependencies.

In [ ]:
# Run common utilities notebook (optional, can be skipped if already run in session)
%run 01_toolkit_common.ipynb

# Ensure we have all the necessary imports
import json
import sys
import os
from datetime import datetime

# Add the parent directory to the path so we can import the toolkit
if not os.path.abspath('..') in sys.path:
    sys.path.insert(0, os.path.abspath('..'))

# Import necessary modules directly
from src.adaptive_cards_toolkit.core.element_factory import ElementFactory
from src.adaptive_cards_toolkit.core.layout_helper import LayoutHelper
from src.adaptive_cards_toolkit.templates.templates import TemplateFactory
from src.adaptive_cards_toolkit.core.data_connector import DataConnector

# Define our own validation utility since the imported one might have issues
class SimpleValidator:
    def validate(self, card):
        """Simple validation for cards - just calculates size"""
        try:
            card_json = card.to_json()
            size_kb = len(card_json) / 1024.0
            return {
                "valid": True,
                "size": size_kb,
                "details": [],
                "warnings": []
            }
        except Exception as e:
            return {
                "valid": False,
                "size": 0,
                "details": [str(e)],
                "warnings": []
            }

# Create an instance of our simple validator
simple_validator = SimpleValidator()

# Define display_card function for consistent rendering
def display_card(card, title=None):
    """Display a card as JSON with basic validation.
    
    Args:
        card: The card object to display
        title: Optional title to display with the card
    """
    if title:
        print(f"\n## {title}")
    try:
        # Get JSON from the card
        if hasattr(card, 'to_json'):
            card_json = json.loads(card.to_json())
        elif hasattr(card, 'card_data'):
            card_json = card.card_data
        else:
            card_json = card
            
        # Print the JSON
        print(json.dumps(card_json, indent=2))
        
        # Validate card
        if hasattr(card, 'to_json'):
            validation = simple_validator.validate(card)
            print(f"\nCard size: {validation['size']:.2f}KB | Valid: {validation['valid']}")
    except Exception as e:
        print(f"Error displaying card: {e}")

## Learning Objectives

In this notebook, you'll learn how to:

1. Create documentation page templates with code samples
2. Build news digest templates for multiple stories
3. Implement a complete template library
4. Apply best practices for specialized content templates

## Specialized Content Templates

Let's create templates for more specialized use cases, making sure to avoid the schema-related issues we encountered previously.

### Documentation Page Template

First, let's create a template for technical documentation that can include code samples:

In [ ]:
def create_documentation_page_template(title, content, metadata=None, code_samples=None, actions=None):
    """Create a documentation page template with code samples.
    
    Args:
        title (str): Document title
        content (str): Main content text
        metadata (dict, optional): Dictionary of metadata (version, updated, category, etc.)
        code_samples (list, optional): List of code sample dictionaries with {language, title, code} keys
        actions (list, optional): List of action dictionaries
        
    Returns:
        Object: Card object with to_json method
    """
    try:
        # Create card data with JSON construction pattern from style guide
        card_data = {
            "type": "AdaptiveCard",
            "version": "1.5",
            "body": []
        }
        
        # Add title
        card_data["body"].append({
            "type": "TextBlock",
            "text": title,
            "size": "extraLarge",
            "weight": "bolder",
            "wrap": True
        })
        
        # Add metadata if provided
        if metadata and isinstance(metadata, dict):
            metadata_text = []
            
            if 'version' in metadata:
                metadata_text.append(f"Version: {metadata['version']}")
                
            if 'updated' in metadata:
                metadata_text.append(f"Updated: {metadata['updated']}")
                
            if 'category' in metadata:
                metadata_text.append(f"Category: {metadata['category']}")
                
            if metadata_text:
                card_data["body"].append({
                    "type": "TextBlock",
                    "text": " | ".join(metadata_text),
                    "isSubtle": True,
                    "size": "small",
                    "wrap": True
                })
        
        # Add content with paragraph spacing
        paragraphs = content.split('\n\n')
        for paragraph in paragraphs:
            card_data["body"].append({
                "type": "TextBlock",
                "text": paragraph,
                "wrap": True
            })
        
        # Add code samples if provided
        if code_samples and isinstance(code_samples, list):
            for sample in code_samples:
                # Create a container for the code sample
                code_container = {
                    "type": "Container",
                    "style": "emphasis",
                    "spacing": "medium",
                    "items": []
                }
                
                if 'language' in sample:
                    code_container["items"].append({
                        "type": "TextBlock",
                        "text": f"Language: {sample['language']}",
                        "size": "small",
                        "isSubtle": True,
                        "wrap": True
                    })
                
                if 'title' in sample:
                    code_container["items"].append({
                        "type": "TextBlock",
                        "text": sample['title'],
                        "weight": "bolder",
                        "wrap": True
                    })
                
                if 'code' in sample:
                    code_container["items"].append({
                        "type": "TextBlock",
                        "text": sample['code'],
                        "fontType": "monospace",
                        "wrap": True
                    })
                
                card_data["body"].append(code_container)
        
        # Add actions if provided
        if actions:
            card_data["actions"] = []
            for action in actions:
                if action.get('type') == 'open_url':
                    card_data["actions"].append({
                        "type": "Action.OpenUrl",
                        "title": action.get('title', 'Open'),
                        "url": action.get('url', '#')
                    })
                elif action.get('type') == 'submit':
                    card_data["actions"].append({
                        "type": "Action.Submit",
                        "title": action.get('title', 'Submit'),
                        "data": action.get('data', {})
                    })
        
        # Create a custom card object with to_json method
        class CardObject:
            def __init__(self, card_data):
                self.card_data = card_data
                
            def to_json(self):
                return json.dumps(self.card_data)
        
        return CardObject(card_data)
    
    except Exception as e:
        print(f"Error creating documentation page template: {e}")
        return None

Let's test our documentation page template:

In [ ]:
try:
    # Create a documentation page
    doc_page = create_documentation_page_template(
        title="Getting Started with Adaptive Cards Toolkit",
        content="The Adaptive Cards Toolkit provides a high-level API for creating and managing adaptive cards for AI agents and applications.\n\nThis guide will walk you through the basic setup and usage of the toolkit's core components.",
        metadata={
            "version": "1.2.0",
            "updated": "February 15, 2023",
            "category": "Getting Started"
        },
        code_samples=[
            {
                "language": "Python",
                "title": "Installation",
                "code": "pip install adaptive-cards-toolkit"
            },
            {
                "language": "Python",
                "title": "Basic Usage",
                "code": "from adaptive_cards_toolkit import AgentCardBuilder\n\nbuilder = AgentCardBuilder()\ncard = builder.create_basic_card(\n    title=\"Hello World\",\n    message=\"This is my first adaptive card.\"\n)"
            }
        ],
        actions=[
            {
                "type": "open_url",
                "title": "Full Documentation",
                "url": "https://adaptive-cards-toolkit.readthedocs.io/"
            }
        ]
    )
    
    display_card(doc_page, "Documentation Page")
    
except Exception as e:
    print(f"Error creating documentation page: {e}")

### News Digest Template

Now let's create a template for news digests that can display multiple stories in a compact format:

In [ ]:
def create_news_digest_template(title, news_items, date=None, actions=None):
    """Create a news digest template with multiple news items.
    
    Args:
        title (str): Digest title
        news_items (list): List of news item dictionaries with {title, summary, source, date, image_url, url} keys
        date (str, optional): Publication date of the digest
        actions (list, optional): List of action dictionaries
        
    Returns:
        Object: Card object with to_json method
    """
    try:
        # Create card data with JSON construction pattern
        card_data = {
            "type": "AdaptiveCard",
            "version": "1.5",
            "body": []
        }
        
        # Add title
        card_data["body"].append({
            "type": "TextBlock",
            "text": title,
            "size": "extraLarge",
            "weight": "bolder",
            "wrap": True
        })
        
        # Add date if provided
        if date:
            card_data["body"].append({
                "type": "TextBlock",
                "text": f"Published: {date}",
                "isSubtle": True,
                "wrap": True
            })
        
        # Add each news item
        for i, item in enumerate(news_items):
            # Add separator between items (except before the first item)
            if i > 0:
                card_data["body"].append({
                    "type": "Separator"
                })
            
            # Create container for the news item
            item_container = {
                "type": "Container",
                "spacing": "medium",
                "items": []
            }
            
            # Create column set for layout
            column_set = {
                "type": "ColumnSet",
                "columns": []
            }
            
            # Left column for content
            left_column = {
                "type": "Column",
                "width": "stretch",
                "items": []
            }
            
            # Add news item title
            if 'title' in item:
                left_column["items"].append({
                    "type": "TextBlock",
                    "text": item['title'],
                    "weight": "bolder",
                    "size": "medium",
                    "wrap": True
                })
            
            # Add news item summary
            if 'summary' in item:
                left_column["items"].append({
                    "type": "TextBlock",
                    "text": item['summary'],
                    "wrap": True
                })
            
            # Add source and date
            source_info = []
            if 'source' in item:
                source_info.append(item['source'])
            if 'date' in item:
                source_info.append(item['date'])
            
            if source_info:
                left_column["items"].append({
                    "type": "TextBlock",
                    "text": " | ".join(source_info),
                    "isSubtle": True,
                    "size": "small",
                    "wrap": True
                })
            
            column_set["columns"].append(left_column)
            
            # Right column for image (if provided)
            if 'image_url' in item:
                right_column = {
                    "type": "Column",
                    "width": "auto",
                    "items": [
                        {
                            "type": "Image",
                            "url": item['image_url'],
                            "altText": f"Image for {item.get('title', 'news item')}",
                            "size": "small"
                        }
                    ]
                }
                column_set["columns"].append(right_column)
            
            # Add column set to container
            item_container["items"].append(column_set)
            
            # Add "Read more" link if URL provided
            if 'url' in item:
                item_container["items"].append({
                    "type": "TextBlock",
                    "text": f"[Read more]({item['url']})",
                    "size": "small",
                    "wrap": True
                })
            
            # Add the item container to the card
            card_data["body"].append(item_container)
        
        # Add actions if provided
        if actions:
            card_data["actions"] = []
            for action in actions:
                if action.get('type') == 'open_url':
                    card_data["actions"].append({
                        "type": "Action.OpenUrl",
                        "title": action.get('title', 'Open'),
                        "url": action.get('url', '#')
                    })
                elif action.get('type') == 'submit':
                    card_data["actions"].append({
                        "type": "Action.Submit",
                        "title": action.get('title', 'Submit'),
                        "data": action.get('data', {})
                    })
        
        # Create a custom card object with to_json method
        class CardObject:
            def __init__(self, card_data):
                self.card_data = card_data
                
            def to_json(self):
                return json.dumps(self.card_data)
        
        return CardObject(card_data)
    
    except Exception as e:
        print(f"Error creating news digest template: {e}")
        return None

Let's test our news digest template:

In [ ]:
try:
    # Create a news digest
    news_digest = create_news_digest_template(
        title="AI Weekly Digest",
        date="February 22, 2023",
        news_items=[
            {
                "title": "New Research Shows AI-Generated UI Increases User Engagement",
                "summary": "A recent study by Adaptive Intelligence Labs found that AI-generated user interfaces led to a 27% increase in user engagement compared to static UIs.",
                "source": "AI Today",
                "date": "February 20, 2023",
                "image_url": "https://adaptivecards.io/content/adaptive-card-50.png",
                "url": "https://example.com/ai-ui-research"
            },
            {
                "title": "Microsoft Expands Adaptive Cards Support in Teams",
                "summary": "Microsoft announced expanded support for Adaptive Cards in Teams, including new components and improved rendering performance.",
                "source": "Tech News",
                "date": "February 18, 2023",
                "image_url": "https://adaptivecards.io/content/adaptive-card-50.png",
                "url": "https://example.com/ms-teams-cards"
            },
            {
                "title": "Open Source Toolkit for AI-Generated UI Released",
                "summary": "A new open-source toolkit aims to standardize how AI agents generate user interfaces, with a focus on accessibility and cross-platform support.",
                "source": "Open Source Weekly",
                "date": "February 16, 2023",
                "url": "https://example.com/ai-ui-toolkit"
            }
        ],
        actions=[
            {
                "type": "open_url",
                "title": "View All News",
                "url": "https://example.com/ai-news"
            },
            {
                "type": "submit",
                "title": "Subscribe to Digest",
                "data": {"action": "subscribe_digest"}
            }
        ]
    )
    
    display_card(news_digest, "News Digest")
    
except Exception as e:
    print(f"Error creating news digest: {e}")

### Tutorial Template with Steps

Let's create a template for step-by-step tutorials:

In [ ]:
def create_tutorial_template(title, intro, steps, conclusion=None, metadata=None, actions=None):
    """Create a tutorial template with numbered steps.
    
    Args:
        title (str): Tutorial title
        intro (str): Introduction text
        steps (list): List of step dictionaries with {title, content, image_url, code} keys
        conclusion (str, optional): Conclusion text
        metadata (dict, optional): Dictionary of metadata (author, difficulty, time, etc.)
        actions (list, optional): List of action dictionaries
        
    Returns:
        Object: Card object with to_json method
    """
    try:
        # Create card data with JSON construction pattern
        card_data = {
            "type": "AdaptiveCard",
            "version": "1.5",
            "body": []
        }
        
        # Add title
        card_data["body"].append({
            "type": "TextBlock",
            "text": title,
            "size": "extraLarge",
            "weight": "bolder",
            "wrap": True
        })
        
        # Add metadata if provided
        if metadata and isinstance(metadata, dict):
            metadata_container = {
                "type": "Container",
                "style": "emphasis",
                "items": []
            }
            
            metadata_items = []
            
            if 'author' in metadata:
                metadata_items.append(f"Author: {metadata['author']}")
            
            if 'difficulty' in metadata:
                metadata_items.append(f"Difficulty: {metadata['difficulty']}")
                
            if 'time' in metadata:
                metadata_items.append(f"Time: {metadata['time']}")
                
            if 'prerequisites' in metadata:
                prereqs = metadata['prerequisites']
                if isinstance(prereqs, list):
                    metadata_items.append(f"Prerequisites: {', '.join(prereqs)}")
                else:
                    metadata_items.append(f"Prerequisites: {prereqs}")
            
            if metadata_items:
                metadata_container["items"].append({
                    "type": "TextBlock",
                    "text": " | ".join(metadata_items),
                    "wrap": True
                })
                
                card_data["body"].append(metadata_container)
        
        # Add introduction
        if intro:
            card_data["body"].append({
                "type": "TextBlock",
                "text": intro,
                "wrap": True
            })
        
        # Add steps heading
        card_data["body"].append({
            "type": "TextBlock",
            "text": "Tutorial Steps",
            "size": "large",
            "weight": "bolder",
            "spacing": "medium",
            "wrap": True
        })
        
        # Add each step
        for i, step in enumerate(steps):
            step_number = i + 1
            
            # Create step container
            step_container = {
                "type": "Container",
                "style": "default",
                "spacing": "medium",
                "items": []
            }
            
            # Step title with number
            step_title = f"Step {step_number}: {step.get('title', 'Complete this step')}"
            step_container["items"].append({
                "type": "TextBlock",
                "text": step_title,
                "size": "medium",
                "weight": "bolder",
                "wrap": True
            })
            
            # Step content
            if 'content' in step:
                step_container["items"].append({
                    "type": "TextBlock",
                    "text": step['content'],
                    "wrap": True
                })
            
            # Step image if provided
            if 'image_url' in step:
                step_container["items"].append({
                    "type": "Image",
                    "url": step['image_url'],
                    "altText": f"Image for step {step_number}",
                    "size": "medium"
                })
            
            # Step code if provided
            if 'code' in step:
                code_container = {
                    "type": "Container",
                    "style": "emphasis",
                    "items": [
                        {
                            "type": "TextBlock",
                            "text": step.get('code_title', 'Code'),
                            "size": "small",
                            "weight": "bolder",
                            "wrap": True
                        },
                        {
                            "type": "TextBlock",
                            "text": step['code'],
                            "fontType": "monospace",
                            "wrap": True
                        }
                    ]
                }
                step_container["items"].append(code_container)
            
            # Add the step container to the card
            card_data["body"].append(step_container)
        
        # Add conclusion if provided
        if conclusion:
            card_data["body"].append({
                "type": "TextBlock",
                "text": "Conclusion",
                "size": "medium",
                "weight": "bolder",
                "spacing": "medium",
                "wrap": True
            })
            
            card_data["body"].append({
                "type": "TextBlock",
                "text": conclusion,
                "wrap": True
            })
        
        # Add actions if provided
        if actions:
            card_data["actions"] = []
            for action in actions:
                if action.get('type') == 'open_url':
                    card_data["actions"].append({
                        "type": "Action.OpenUrl",
                        "title": action.get('title', 'Open'),
                        "url": action.get('url', '#')
                    })
                elif action.get('type') == 'submit':
                    card_data["actions"].append({
                        "type": "Action.Submit",
                        "title": action.get('title', 'Submit'),
                        "data": action.get('data', {})
                    })
        
        # Create a custom card object with to_json method
        class CardObject:
            def __init__(self, card_data):
                self.card_data = card_data
                
            def to_json(self):
                return json.dumps(self.card_data)
        
        return CardObject(card_data)
    
    except Exception as e:
        print(f"Error creating tutorial template: {e}")
        return None

Let's test our tutorial template:

In [ ]:
try:
    # Create a tutorial
    tutorial = create_tutorial_template(
        title="Creating Your First Adaptive Card",
        intro="This tutorial will walk you through the process of creating a basic adaptive card using the Adaptive Cards Toolkit.",
        metadata={
            "author": "Adaptive Cards Team",
            "difficulty": "Beginner",
            "time": "10 minutes",
            "prerequisites": ["Python 3.6+", "pip"]
        },
        steps=[
            {
                "title": "Install the Toolkit",
                "content": "First, install the Adaptive Cards Toolkit using pip.",
                "code": "pip install adaptive-cards-toolkit"
            },
            {
                "title": "Import Required Modules",
                "content": "Import the necessary modules to create your card.",
                "code": "from adaptive_cards_toolkit import AgentCardBuilder\nfrom adaptive_cards_toolkit.element_factory import ElementFactory"
            },
            {
                "title": "Create a Basic Card",
                "content": "Use the AgentCardBuilder to create a simple card with a title and message.",
                "code": "builder = AgentCardBuilder()\n\ncard = builder.create_basic_card(\n    title=\"Hello World\",\n    message=\"This is my first adaptive card.\"\n)",
                "image_url": "https://adaptivecards.io/content/adaptive-card-50.png"
            },
            {
                "title": "Convert to JSON",
                "content": "Convert the card to JSON format for delivery to your target platform.",
                "code": "card_json = builder.get_json(card)\nprint(card_json)"
            }
        ],
        conclusion="Congratulations! You've created your first adaptive card. You can now customize it further by adding more elements or actions to make it interactive.",
        actions=[
            {
                "type": "open_url",
                "title": "View Documentation",
                "url": "https://adaptive-cards-toolkit.readthedocs.io/"
            },
            {
                "type": "submit",
                "title": "Next Tutorial",
                "data": {"action": "next_tutorial", "id": "adding-actions"}
            }
        ]
    )
    
    display_card(tutorial, "Tutorial Card")
    
except Exception as e:
    print(f"Error creating tutorial: {e}")

## Complete Content Template Library

Let's create a content template library class that brings together all our templates:

In [ ]:
class ContentTemplateLibrary:
    """A comprehensive library of content templates that uses direct JSON construction.
    
    This library avoids using AdaptiveCard.create() and schema-related methods to prevent issues.
    """
    
    def __init__(self):
        """Initialize the content template library."""
        pass
    
    def create_article(self, title, content, image_url=None, author=None, date=None, actions=None):
        """Create a basic article template.
        
        Args:
            title (str): Article title
            content (str): Article content
            image_url (str, optional): URL to a header image
            author (str, optional): Article author
            date (str, optional): Publication date
            actions (list, optional): List of action dictionaries
            
        Returns:
            Object: Card object with to_json method
        """
        # Create card data with JSON construction pattern
        card_data = {
            "type": "AdaptiveCard",
            "version": "1.5",
            "body": []
        }
        
        # Add header image if provided
        if image_url:
            card_data["body"].append({
                "type": "Image",
                "url": image_url,
                "altText": f"Image for {title}",
                "size": "stretch"
            })
        
        # Add title
        card_data["body"].append({
            "type": "TextBlock",
            "text": title,
            "size": "extraLarge",
            "weight": "bolder",
            "wrap": True
        })
        
        # Add author and date if provided
        metadata = []
        if author:
            metadata.append(f"By {author}")
        if date:
            metadata.append(f"Published: {date}")
            
        if metadata:
            metadata_text = " · ".join(metadata)
            card_data["body"].append({
                "type": "TextBlock",
                "text": metadata_text,
                "isSubtle": True,
                "wrap": True
            })
            
        # Add content with paragraph spacing
        paragraphs = content.split('\n\n')
        for paragraph in paragraphs:
            card_data["body"].append({
                "type": "TextBlock",
                "text": paragraph,
                "wrap": True
            })
        
        # Add actions if provided
        if actions:
            card_data["actions"] = []
            for action in actions:
                if action.get('type') == 'open_url':
                    card_data["actions"].append({
                        "type": "Action.OpenUrl",
                        "title": action.get('title', 'Open'),
                        "url": action.get('url', '#')
                    })
                elif action.get('type') == 'submit':
                    card_data["actions"].append({
                        "type": "Action.Submit",
                        "title": action.get('title', 'Submit'),
                        "data": action.get('data', {})
                    })
        
        # Create and return a card object
        class CardObject:
            def __init__(self, card_data):
                self.card_data = card_data
                
            def to_json(self):
                return json.dumps(self.card_data)
        
        return CardObject(card_data)
    
    def create_documentation_page(self, title, content, metadata=None, code_samples=None, actions=None):
        """Create a documentation page template."""
        return create_documentation_page_template(
            title=title,
            content=content,
            metadata=metadata,
            code_samples=code_samples,
            actions=actions
        )
    
    def create_news_digest(self, title, news_items, date=None, actions=None):
        """Create a news digest template with multiple news items."""
        return create_news_digest_template(
            title=title,
            news_items=news_items,
            date=date,
            actions=actions
        )
    
    def create_tutorial(self, title, intro, steps, conclusion=None, metadata=None, actions=None):
        """Create a tutorial template with numbered steps."""
        return create_tutorial_template(
            title=title,
            intro=intro,
            steps=steps,
            conclusion=conclusion,
            metadata=metadata,
            actions=actions
        )
        
    def create_changelog(self, title, version, date, changes, previous_versions=None, actions=None):
        """Create a changelog template for displaying updates.
        
        Args:
            title (str): Changelog title
            version (str): Current version number
            date (str): Release date
            changes (dict): Dictionary with keys like 'new', 'improved', 'fixed' and list values
            previous_versions (list, optional): List of previous version dictionaries
            actions (list, optional): List of action dictionaries
            
        Returns:
            Object: Card object with to_json method
        """
        # Create card data with JSON construction pattern
        card_data = {
            "type": "AdaptiveCard",
            "version": "1.5",
            "body": []
        }
        
        # Add title
        card_data["body"].append({
            "type": "TextBlock",
            "text": title,
            "size": "extraLarge",
            "weight": "bolder",
            "wrap": True
        })
        
        # Add version and date
        card_data["body"].append({
            "type": "TextBlock",
            "text": f"Version {version} - {date}",
            "size": "medium",
            "weight": "bolder",
            "wrap": True
        })
        
        # Add changes by category
        for category, items in changes.items():
            if items and len(items) > 0:
                # Add category header
                card_data["body"].append({
                    "type": "TextBlock",
                    "text": category.capitalize(),
                    "weight": "bolder",
                    "wrap": True
                })
                
                # Add list of changes
                change_list = ""
                for item in items:
                    change_list += f"- {item}\n"
                
                card_data["body"].append({
                    "type": "TextBlock",
                    "text": change_list,
                    "wrap": True
                })
        
        # Add previous versions if provided
        if previous_versions and len(previous_versions) > 0:
            card_data["body"].append({
                "type": "TextBlock",
                "text": "Previous Versions",
                "size": "medium",
                "weight": "bolder",
                "spacing": "medium",
                "wrap": True
            })
            
            for prev in previous_versions:
                # Add container for previous version
                prev_container = {
                    "type": "Container",
                    "style": "emphasis",
                    "items": [
                        {
                            "type": "TextBlock",
                            "text": f"Version {prev.get('version', '?')} - {prev.get('date', '')}",
                            "weight": "bolder",
                            "wrap": True
                        }
                    ]
                }
                
                # Add summary if provided
                if 'summary' in prev:
                    prev_container["items"].append({
                        "type": "TextBlock",
                        "text": prev['summary'],
                        "wrap": True
                    })
                
                card_data["body"].append(prev_container)
        
        # Add actions if provided
        if actions:
            card_data["actions"] = []
            for action in actions:
                if action.get('type') == 'open_url':
                    card_data["actions"].append({
                        "type": "Action.OpenUrl",
                        "title": action.get('title', 'Open'),
                        "url": action.get('url', '#')
                    })
                elif action.get('type') == 'submit':
                    card_data["actions"].append({
                        "type": "Action.Submit",
                        "title": action.get('title', 'Submit'),
                        "data": action.get('data', {})
                    })
        
        # Create and return a card object
        class CardObject:
            def __init__(self, card_data):
                self.card_data = card_data
                
            def to_json(self):
                return json.dumps(self.card_data)
        
        return CardObject(card_data)

Let's test our ContentTemplateLibrary with a changelog template:

In [ ]:
try:
    # Create template library instance
    template_library = ContentTemplateLibrary()
    
    # Create a changelog
    changelog = template_library.create_changelog(
        title="Adaptive Cards Toolkit Changelog",
        version="1.3.0",
        date="March 5, 2023",
        changes={
            "new": [
                "Added support for direct JSON card construction",
                "New specialized templates for documentation, tutorials, and news digests",
                "Support for monospace text in code samples"
            ],
            "improved": [
                "Better error handling in template functions",
                "More flexible metadata display options",
                "Enhanced validation utility with size estimation"
            ],
            "fixed": [
                "Resolved schema validation errors in complex templates",
                "Fixed wrapping issues in text blocks",
                "Corrected image sizing in mobile layouts"
            ]
        },
        previous_versions=[
            {
                "version": "1.2.0",
                "date": "February 15, 2023",
                "summary": "Added profile templates and improved form components"
            },
            {
                "version": "1.1.0",
                "date": "January 20, 2023",
                "summary": "Added data visualization support and dashboard templates"
            },
            {
                "version": "1.0.0",
                "date": "January 5, 2023",
                "summary": "Initial release with basic card creation functionality"
            }
        ],
        actions=[
            {
                "type": "open_url",
                "title": "Release Notes",
                "url": "https://example.com/release-notes"
            },
            {
                "type": "submit",
                "title": "Update Now",
                "data": {"action": "update_toolkit"}
            }
        ]
    )
    
    display_card(changelog, "Changelog Card")
    
except Exception as e:
    print(f"Error creating changelog: {e}")

## Note on Common Errors

When implementing Adaptive Cards with this toolkit, be aware of these common errors:

1. **'property' object has no attribute 'schema'** - This appears in AdaptiveCard related operations, specifically when creating cards with AdaptiveCard.create(). The fix is to:
   - Use direct JSON construction as shown in these examples
   - Create a CardObject wrapper with to_json() method
   - Avoid using AdaptiveCard.create() or schema-related methods

2. **ElementFactory.create_text() got an unexpected keyword argument 'wrap'** - The ElementFactory methods don't accept a 'wrap' parameter directly. Instead:
   - Use direct JSON construction as shown in the examples above
   - Set "wrap": True directly in the JSON structure

3. **'NoneType' object has no attribute 'to_json'** - This occurs when card creation fails and returns None, but code tries to use to_json(). To fix:
   - Use a CardObject wrapper with a to_json() method as shown in these examples
   - Ensure exception handling properly checks for None before calling to_json()
   - Follow the Card Creation Pattern from CLAUDE.md that uses direct JSON construction for complex cards

These examples use direct JSON construction to avoid these errors. This approach follows the recommended pattern in the toolkit.

## Key Takeaways

In this notebook, you've learned:

1. How to create specialized documentation templates with code samples
2. How to build news digest templates for multiple stories
3. How to create tutorial templates with numbered steps
4. How to implement a complete template library for content
5. How to use direct JSON construction to avoid schema-related errors

These specialized templates demonstrate the flexibility of Adaptive Cards for creating rich, structured content for a variety of use cases.